У меня пока что очень мало репрезентативного кода, но я горжусь функцией ниже.

Это функция для подготовки датасета из первого проекта специализации,
и я ускорил её на датасете из 150 пользователей с ~33 минут до ~2-3 секунд, в зависимости от машины.

К сожалению, самый медленный сохранившийся код выполняетсяс около 20-30 секунд, поэтому для сравнения выложу его.

Меня очень порадовала эта задача и то, что я сделал, поэтому я выкладываю именно этот код.

In [26]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from collections import defaultdict
from csv import reader as cr

from time import time

PATH_TO_DATA = '.' 

In [24]:
#~20 sec version
def prepare_train_set(path_to_csv_files, session_length=10):   
    tme = time()
    u_sites = dict()
    uid = 1
    df = []
    columns=['timestamp', 'site', 'user_id']
    framepaths = glob(path_to_csv_files + '\\*.csv')
    for framepath in framepaths:
        with open(framepath) as file:
            frame = list(cr(file))
        for item in frame:
            u_sites[item[1]] = u_sites.get(item[1], 0) + 1
            item.append(uid)
            df.append(item)
        uid += 1
    df = pd.DataFrame(df, columns=columns)
    u_sites = {kv[0]: (i, kv[1]) for i, kv in enumerate(sorted(u_sites.items(), 
                                                      key=lambda item: item[1], 
                                                      reverse=True), start=1)}

    columns = ['site' + str(i) for i in range(1, session_length + 1)]
    columns.append('user_id')
    
    compact_frame = []
    users = df.user_id.unique()
    df['site_id'] = df.site.apply(lambda x: u_sites[x][0])
    for uid in users:
        cur_df = df[df.user_id==uid]
        tens = session_length
        while tens <= cur_df.shape[0] + session_length:
            row = list(cur_df.site_id[tens-session_length:tens])
            while len(row) < session_length:
                row.append(0)
            row.append(uid)
            compact_frame.append(row)
            tens += session_length
    print('\n', time() - tme, sep='')        
    return pd.DataFrame(compact_frame, columns=columns), u_sites

In [14]:
#~2 sec version
def prepare_train_set_new(path_to_csv_files, session_length=10):   
    tme = time()
    uid = 1
    u_sites = dict()
    compact_frame = []
    
    framepaths = glob(path_to_csv_files + os.sep + '*.csv')
    for framepath in framepaths:
        frame = pd.read_csv(framepath)
        frame = list(frame['site'].values)
        for i in range(0, len(frame), session_length):
            row = frame[i : i + session_length]
            for key in row:
                u_sites[key] = u_sites.get(key, 0) + 1
            k = session_length - len(row)
            if k > 0:
                zeros = [0] * k
                row.extend(zeros)
            row.append(uid)
            compact_frame.append(row)
        uid += 1
    
    u_sites = {kv[0]: (i, kv[1]) for i, kv in enumerate(sorted(u_sites.items(), 
                                                      key=lambda item: item[1], 
                                                      reverse=True), start=1)}

    columns = ['site' + str(i) for i in range(1, session_length + 1)]
    columns.append('user_id')
    
    compact_frame = pd.DataFrame(compact_frame, columns=columns)
    
    for item in columns[:-1]:
        compact_frame[item] = compact_frame[item].apply(lambda x: u_sites.get(x, (0, 0))[0])
    print('\n', time() - tme, sep='')     
    return compact_frame, u_sites


In [8]:
#%prun       tme = time()      print('\n', time() - tme, sep='')     
dff, uniques = prepare_train_set_new(os.path.join(PATH_TO_DATA, '10users'), 10)


0.2898440361022949


In [9]:
dff, uniques = prepare_train_set_new(os.path.join(PATH_TO_DATA, '3users'), 10)


0.014989376068115234


In [11]:
#%%time
train_data_150users, site_freq_150users = prepare_train_set_new(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)


3.018272638320923


In [21]:
#%prun 
dff, uniques = prepare_train_set(os.path.join(PATH_TO_DATA, '10users'), 10)


2.420649528503418


In [22]:
dff, uniques = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 10)


0.0069963932037353516


In [23]:
#%%time
train_data_150users, site_freq_150users = prepare_train_set(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)


23.329655408859253
